In [93]:
cd F:\ying\CPP - AD\M4_data\m4-saline1

F:\ying\CPP - AD\M4_data\m4-saline1


In [94]:
from scipy.signal import butter, lfilter, freqz
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
lowcut = 0.5
highcut = 10
#y = butter_bandpass_filter(x, lowcut, highcut, fs, order=4)

In [95]:
import numpy as np
import math
from collections import defaultdict
import matplotlib.pyplot as plt
from scipy.signal import find_peaks_cwt


data = dict(np.load('m4-saline1-cont.npz'))
data.keys()
raw_trace = data['ica_mixing'][:, data['ica_selected_components']]

In [96]:
data['start_frames']
data['total_frames']
temp = np.absolute(np.diff(data['timestamp']))
time_mean = np.mean(temp)
a = [i for i,v in enumerate(temp) if v > time_mean*15]
a = [i+1 for i in a]

In [97]:
#raw_pre = data['ica_mixing'][20*12:150*20, data['ica_selected_components']]
#time_pre = data['timestamp'][20*12:150*20]
raw_pre = data['ica_mixing'][math.ceil(data['start_frames'][0]*7.5):150*20+math.ceil(data['start_frames'][0]*7.5), data['ica_selected_components']]
raw_session = data['ica_mixing'][a[0]+math.ceil(data['start_frames'][1]*7.5):280*20+a[0]+math.ceil(data['start_frames'][1]*7.5),data['ica_selected_components']]
raw_post = data['ica_mixing'][a[1]+math.ceil(data['start_frames'][2]*7.5):150*20+a[1]+math.ceil(data['start_frames'][2]*7.5),data['ica_selected_components']]

#raw_post = data['ica_mixing'][a[2]:,data['ica_selected_components']]
time_pre = data['timestamp'][math.ceil(data['start_frames'][0]*7.5):150*20+math.ceil(data['start_frames'][0]*7.5)]
time_session = data['timestamp'][a[0]+math.ceil(data['start_frames'][1]*7.5):280*20+a[0]+math.ceil(data['start_frames'][1]*7.5)]
time_post = data['timestamp'][a[1]+math.ceil(data['start_frames'][2]*7.5):150*20 + a[1]+math.ceil(data['start_frames'][2]*7.5)]


mean = np.mean(raw_pre, axis=1)
median = np.median(raw_pre, axis=1)
stdev = np.std(raw_pre, axis=1)

#diff = np.diff(raw_pre, axis=0)

disc_pre= np.zeros_like(raw_pre)
spike_no = np.zeros(raw_pre.shape[1])
i = 25
cutoff = 0.3
tolerance = 0.05
filtered = np.array([x for x in raw_pre[:,i] if x<cutoff])    
filtered_stdev = 3*np.std(filtered)
print(cutoff, filtered_stdev)
while abs(cutoff-filtered_stdev) > tolerance:
    if cutoff > filtered_stdev:
        cutoff = cutoff*0.9
    else:
        cutoff = cutoff*1.1
    filtered = np.array([x for x in raw_pre[:,i] if x<cutoff])
    plt.plot(filtered)
    plt.show()
    filtered_stdev = 3*np.std(filtered)
    print(cutoff, filtered_stdev)    

In [98]:
# this is how chen found transients
def find_transient(x, method = 1):
    ## method 1 chen's method of finding cutoff
    ## method 2 Desseiroth's paper of finding cutoff
    mask_up = np.zeros_like(x)
    zeros_and_ones = mask_up
    n= 0
    peaks = []
    diff = np.subtract(x[2:],x[:-2])
    spike_length = 14
    spike_rise = 0.02
    if method ==1:    
        median = np.median(x)
        mad = np.median(np.abs(x - median))
        cutoff = median + 2*mad
        j = spike_length
        while j < len(x)-spike_length:
            if x[i]>cutoff and max(diff[j-spike_length:j+spike_length])> spike_rise:                                             
                n+=1
                mask_up[j:j+spike_length] = x[j:j+spike_length]
                zeros_and_ones[j:j+spike_length] = 1
                j =j+spike_length-1
                peaks = peaks+[max(x[j:j+spike_length])]
            else:
                j+=1
    else:
        cutoff = 0.3
        tolerance = 0.03
        filtered = np.array([xi for xi in x if xi<cutoff])    
        filtered_stdev = 3*np.std(filtered)
        while abs(cutoff-filtered_stdev) > tolerance:
            if cutoff > filtered_stdev:
                cutoff = cutoff*0.9
            else:
                cutoff = cutoff*1.1
            filtered = np.array([xi for xi in x if xi<cutoff])    
            filtered_stdev = 3*np.std(filtered)
        
        cutoff = cutoff*2.8
        offset = 2*filtered_stdev
        j = spike_length            
        while j < len(x)-spike_length:
            if x[j]>cutoff and max(diff[j-spike_length:j+spike_length])> spike_rise:                                             
                n+=1
                mask_up[j:j+spike_length] = x[j:j+spike_length]
                zeros_and_ones[j:j+spike_length] = 1
                j =j+spike_length-1
                peaks = peaks+[max(x[j:j+spike_length])]
            else:
                j+=1
        j = spike_length 
        ########### Uses offset technique #############
        #while j < len(x)-spike_length:
        #    if x[j]>onset and max(diff[j-spike_length:j+spike_length])> spike_rise:                                 
        #        #peaks[i].append(max(raw_pre[j:j+spike_length,i]))
        #        peaks[j,i] = raw_pre[j,i]  
        #        j+=1
        #    while raw_pre[j,i]>offset and j<len(raw_pre[:,i])-1:
        #        peaks[j,i] = raw_pre[j,i]
        #        j+=1
        #else:
        #    j+=1
    return mask_up, n, peaks, zeros_and_ones

processed_pre= np.zeros_like(raw_pre)
zeros_and_ones = processed_pre
fire_number = np.zeros(raw_pre.shape[1])
peaks = defaultdict(list)
for i in range(raw_pre.shape[1]):
    processed_pre[:,i], fire_number[i], peaks[i], zeros_and_ones[:,i]= find_transient(raw_pre[:,i], method = 2)
    #for j in range(raw_pre.shape[0]//300):
    #    mask_up[j*300:j*300+300,i] = find_transient(raw_pre[j*300:j*300+300,i])


C:\Miniconda3\lib\site-packages\numpy\core\_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
C:\Miniconda3\lib\site-packages\numpy\core\_methods.py:94: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Miniconda3\lib\site-packages\numpy\core\_methods.py:116: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [99]:
plt.subplot(4,2,1)
plt.plot(raw_pre[:,0])
plt.subplot(4,2,3)
plt.plot(peaks_pre[:,0])
plt.subplot(4,2,2)
plt.plot(raw_pre[:,5])
plt.subplot(4,2,4)
plt.plot(peaks_pre[:,5])
plt.subplot(4,2,5)
plt.plot(raw_pre[:,15])
plt.subplot(4,2,7)
plt.plot(peaks_pre[:,15])
plt.subplot(4,2,6)
plt.plot(raw_pre[:,27])
plt.subplot(4,2,8)
plt.plot(peaks_pre[:,27])
plt.show()

In [100]:
from statistics import mean
print('the number of cells: ', raw_pre.shape[1])
print('average activity of cells: ')
avg_peak = [np.mean(np.array(v)) for k,v in peaks.items()]
avg_peak = np.nan_to_num(avg_peak)
cells = [i for i in range(0,raw_pre.shape[1])]

plt.hist(fire_number,bins='auto')
plt.show()
#plt.bar(cells, avg_peak)
#plt.title('average peak of the cells')
#plt.show()

#plt.bar(cells, fire_number)
#plt.title('average firing rate of the cells')
#plt.show()



the number of cells:  87
average activity of cells: 


C:\Miniconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Miniconda3\lib\site-packages\numpy\core\_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [101]:
############ create a raster plot
plt.subplot(2,1,1)
for i in range(raw_pre.shape[1]):
    plt.scatter(time_pre, np.multiply(zeros_and_ones[:,i],i+1))
plt.show()
#plt.subplot(2,1,2)
#np.sum(disc_pre,axis=0)
#bins = np.linspace(min(time_pre), max(time_pre), 300)
#for j in range(len(bins)):
    


In [102]:
import pandas as pd
import networkx as nx
import scipy.stats as spstat

dataf_pre = pd.DataFrame(processed_pre, columns =data['ica_selected_components'], index = time_pre, dtype=float)

In [112]:
# create correlation data using panda
# lina/justin/patrick's code
def corrmatrx(df1,df2):
    
    """
    Input: dataframe with columns pertaining to raw count data
    
    Output: 1) dataframe of all pair-wise correlations of columns in dataframe
            2) dataframe of all associated p-values  
    
    """
    corr = pd.DataFrame(index=df1.columns, columns=df1.columns, dtype=float)
    pval = pd.DataFrame(index=df1.columns, columns=df1.columns, dtype=float)
    for col1 in df1.columns:
        for col2 in df2.columns:
            corr.loc[col2,col1] = spstat.pearsonr(df1[col1], df2[col2])[0]
            pval.loc[col2,col1] = spstat.pearsonr(df1[col1], df2[col2])[1]
    return [corr, pval]

def threshold_matrix(mcorr, mpval, method='pval', threshold=0.005):
    """
    Input: Correlation matrix, p-value matrix (as dfs)
           Optional: method (pval (default) or corr) and threshold(default 0.005)
           # threshold used by paper is 0.3
    
    Output: Thresholded correlation matrix (as df)
    """
        # pvalue threshold is for 2-sided test, hence we multiple threshold by 2
    # may need to do something else for threshold in the future
    for x in mcorr.columns:
        for y in mcorr.index:
            if method == 'corr':
                if mcorr[x][y] <= threshold:
                    mcorr[x][y] = 0
                #elif mcorr[x][y] < 0 and mcorr[x][y] > -1*threshold:
                 #   mcorr[x][y] = 0
                elif mcorr[x][y] == 1.0:
                    mcorr[x][y] = 0
            elif method == 'pval':
                if mpval[x][y] > 2*threshold:
                    mcorr[x][y] = 0
                elif mcorr[x][y] == 1.0:
                    mcorr[x][y] = 0
                #Remove negative correlations
                elif mcorr[x][y] < 0:
                    mcorr[x][y] = 0
    
    return mcorr

In [115]:
corr_pre,pval_pre = corrmatrx(dataf_pre,dataf_pre)

C:\Miniconda3\lib\site-packages\scipy\stats\stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\Miniconda3\lib\site-packages\scipy\stats\stats.py:5041: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


In [120]:
corr_pre.fillna(0)
threshold_corr_pre = threshold_matrix(corr_pre,pval_pre, method='corr', threshold=0.2 )
threshold_corr_pre.fillna(0)
#heatmap = plt.pcolor(threshold_corr_pre)
#plt.show()

,1,2,3,4,5,7,8,9,10,11,...,83,84,85,86,87,88,90,92,93,110
1,0.000000,0.000000,0.000000,0.00000,0.539543,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.205466,0.0,0.000000,0.0
2,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.238991,0.000000,0.211488,0.0,0.000000,0.0,0.000000,0.0
3,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.331159,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.381021,0.0,0.000000,0.0
4,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
5,0.539543,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
7,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
8,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
9,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
10,0.000000,0.000000,0.331159,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.211695,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
11,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.211695,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [125]:
def df_to_networkx(df):
    """x
    Input: dataframe containing adjacency/correlation matrix
    
    Output: networkx graph with nodes appropriately labelled
    """
    
    G = nx.from_numpy_matrix(df.values)
    
    G = nx.relabel_nodes(G, dict(enumerate(df.columns)))
    
    return G

def get_centrality_measures(G):
    """
    Input: A networkx graph object
    
    Output: A dictionary mapping nodes to centrality measures {node: [x,y,z]}
    and a list indicating the order of centrality measures
    """
    
    #Compute centrality measures
    degrees = nx.degree(G)
    node_strength = nx.degree(G, weight = "weight")
    betweenness = nx.betweenness_centrality(G)
    #eigenvector = nx.eigenvector_centrality(G)
    closeness = nx.closeness_centrality(G)
    communicability = nx.communicability_centrality(G)
    load = nx.load_centrality(G)
    
    centrality_measures_list = ['degree','node_strength','betweenness',
                                # 'eigenvector',
                                'closeness','communicability',
                                'load']    
    
    centrality_measures = {}
    
    for node in G:
        centrality_measures[node] = [degrees[node], node_strength[node],
                            betweenness[node], 
                            # eigenvector[node], 
                            closeness[node],
                            communicability[node], load[node]]            
    
    return [centrality_measures, centrality_measures_list]

In [126]:
network_pre = df_to_networkx(threshold_corr_pre)
centrality_pre, centrality_list_pre = get_centrality_measures(network_pre)

In [127]:
centrality_pre

{1: [16,
  nan,
  0.00010522992739135012,
  0.5512820512820513,
  4.76564078147983,
  0.00010522992739135003],
 2: [16,
  nan,
  7.987452583895811e-05,
  0.5512820512820513,
  14.560566080527856,
  7.987452583895808e-05],
 3: [15,
  nan,
  6.313795643481006e-05,
  0.5477707006369427,
  3.45309085591083,
  6.313795643481002e-05],
 4: [14, nan, 0.0, 0.5443037974683544, 7.490455678701355, 0.0],
 5: [14, nan, 0.0, 0.5443037974683544, 2.9483168704882035, 0.0],
 7: [13, nan, 0.0, 0.5408805031446541, 1.6641050601868863, 0.0],
 8: [14, nan, 0.0, 0.5443037974683544, 11.429040652619706, 0.0],
 9: [13, nan, 0.0, 0.5408805031446541, 1.592914715955457, 0.0],
 10: [16,
  nan,
  0.0001262759128696201,
  0.5512820512820513,
  4.308240010083178,
  0.00012627591286962003],
 11: [15,
  nan,
  6.313795643481006e-05,
  0.5477707006369427,
  4.807093580828548,
  6.313795643481002e-05],
 12: [88,
  nan,
  0.05932138904879709,
  1.0,
  1.0000000000000004,
  0.059321389048797066],
 13: [18,
  nan,
  0.00017214

In [ ]:
# creating 